##Tải dữ liệu và từ điển

In [ ]:
# Import các thư viện cần thiết
from collections import defaultdict
import unicodedata as ud
import pandas as pd
import numpy as np
import math
import ast
import re
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
!git clone https://github.com/undertheseanlp/underthesea

Cloning into 'underthesea'...
remote: Enumerating objects: 7185, done.
remote: Counting objects: 100% (1404/1404), done.
remote: Compressing objects: 100% (863/863), done.
remote: Total 7185 (delta 574), reused 1243 (delta 504), pack-reused 5781
Receiving objects: 100% (7185/7185), 167.46 MiB | 23.37 MiB/s, done.
Resolving deltas: 100% (3669/3669), done.
Updating files: 100% (1606/1606), done.


In [ ]:
# Từ điển
words = open('/content/underthesea/underthesea/corpus/data/Viet74K.txt', encoding='utf-8').readlines()

In [ ]:
new_words = []
for word in words:
    new_words.append(word.replace("-"," "))

In [ ]:
# Tách thành bi_grams và tri_grams
bi_grams = []
tri_grams = []
for word in new_words:
    count_space =0
    for chart in word:
        if chart ==" ":
            count_space += 1
    if count_space == 1:
        bi_grams.append(word.replace("\n",""))
        new_words.remove(word)
    if count_space == 2:
        tri_grams.append(word.replace("\n",""))
        new_words.remove(word)

In [ ]:
len(bi_grams)

27938

In [ ]:
len(tri_grams)

3402

In [ ]:
# Ngữ liệu thu thập
sentences = open('/content/cau.txt', encoding='utf-8').readlines()
tokenize_sentences = [sentence.split(' ') for sentence in sentences]

In [ ]:
print('Số lượng câu đã thu thập:', len(sentences))
sentences[0:2]

Số lượng câu đã thu thập: 95


['Kìa mây mây ngang đầu.\n', 'Kìa núi núi lô nhô.\n']

In [ ]:
max_sentence = max(tokenize_sentences, key=len)
print('Câu có số từ nhiều nhất:', len(max_sentence))
' '.join(max_sentence)

Câu có số từ nhiều nhất: 31


'Hai nhà lãnh đạo cũng trao đổi một số biện pháp nhằm tạo điều kiện thuận lợi cho doanh nghiệp mỗi nước tiếp cận thị trường và đầu tư kinh doanh.\n'

In [ ]:
min_sentence = min(tokenize_sentences, key=len)
print('Câu có số từ ít nhất:', len(min_sentence))
' '.join(min_sentence)

Câu có số từ ít nhất: 5


'Kìa mây mây ngang đầu.\n'

##Thuật toán Longest-Matching


In [ ]:
def syllablize(sentence):
    word_pattern = r'\w+'                    # tương ứng với bất kỳ ký tự từ (chữ cái, chữ số, dấu gạch dưới)
    non_word_pattern = r'[^\w\s]'            # tìm các ký từ không phải từ
    digits_pattern = r'\d+([\.,_]\d+)+'      # tìm các số trong câu

    patterns = [word_pattern, non_word_pattern, digits_pattern] # danh sách các biểu thức chính quy
    combined_pattern = f"({'|'.join(patterns)})"

    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(combined_pattern, sentence, re.UNICODE)  # tìm tất cả chuỗi phù hợp với patterns
    return [token[0] for token in tokens]

In [ ]:
def load_n_grams(path):
    with open(path, encoding='utf8') as f:
        words = f.read()
        words = ast.literal_eval(words)
    return words

In [ ]:
def longest_matching(sentence, bi_grams, tri_grams):
    syllables = syllablize(sentence)
    syl_len = len(syllables)
    curr_id = 0
    word_list = []
    # duyệt qua các âm tiết trong câu
    while curr_id < syl_len:
        curr_word = syllables[curr_id]
        if curr_id >= syl_len - 1:   # kiểm tra âm tiết hiện tại có phải là âm tiết cuối cùng trong câu hay không
            word_list.append(curr_word)  # nếu là âm tiết cuối cùng thì thêm vào word
            break

        next_word = syllables[curr_id + 1]  # âm tiết kế tiếp trong câu
        pair_word = ' '.join([curr_word.lower(), next_word.lower()])

        if curr_id >= (syl_len - 2):
            if pair_word in bi_grams:  # nếu pair_word có trong bi_grams
                word_list.append('_'.join([curr_word, next_word]))   # thì thêm vào word_list bằng cách nối curr_word và next_word bằng dấu gạch dưới
                curr_id += 2
            else:
                word_list.append(curr_word) # âm tiết hiện tại được thêm vào word_list
                curr_id += 1
        else:
            next_next_word = syllables[curr_id + 2]
            triple_word = ' '.join([pair_word, next_next_word.lower()])

            if triple_word in tri_grams:   # nếu triple_word có trong tri_grams
                word_list.append('_'.join([curr_word, next_word, next_next_word]))   # thì thêm vào word_list bằng cách nối curr_word, next_word và next_next_word bằng dấu gạch dưới
                curr_id += 3
            elif pair_word in bi_grams:
                word_list.append('_'.join([curr_word, next_word]))
                curr_id += 2
            else:
                word_list.append(curr_word)
                curr_id += 1
    return word_list

In [ ]:
with open('/content/cau_longestmatching.txt', 'w', encoding='utf-8') as f:
    longest_matching_sentences = []
    for sentence in sentences:
        word_list = longest_matching(sentence, bi_grams, tri_grams)
        longest_matching_sentences.append(' '.join(word_list))
        for word in word_list: f.write(word + '\n')
        if sentence != sentences[-1]: f.write('\n')
    f.write('\n')
longest_matching_compound_words = 0
for sentence in longest_matching_sentences:
    for word in sentence.split():
        if '_' in word: longest_matching_compound_words += 1
print('Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching:', longest_matching_compound_words)
longest_matching_sentences[10:15]

Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching: 186


['Lên trên này thấy các em , anh mới thấy mình quá may_mắn .',
 'Mặt_trời vàng như trứng chiên .',
 'Khi mà em đứng thẳng , núi và đồi phải đứng nghiêng .',
 'Con_đường quanh_co quá , em chao lượn như cánh chim .',
 'Mai_sau lớn em bơi giữa đời , em bơi khỏe như Ánh Viên .']

##Thư viện Underthesea

In [ ]:
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 51.8 MB/s eta 0:00:00


In [ ]:
from underthesea import word_tokenize

In [ ]:
with open('/content/cau_underthesea.txt', 'w', encoding='utf-8') as f:
  underthesea_sentences = []
  for sentence in sentences:
      word_list = word_tokenize(str(sentence).strip(),format="text") #Tách câu thành danh sách các từ bằng cách sử dụng hàm 'word_tokenize'
      word_list = word_list.split(" ")
      underthesea_sentences.append(' '.join(word_list))
      for word in word_list:
        f.write(word + '\n')
      if sentence != sentences[-1]: f.write('\n')
  f.write('\n')
underthesea_compound_words = 0
for sentence in underthesea_sentences:
    for word in sentence:
        if '_' in word:
          underthesea_compound_words += 1
print('Số lượng từ ghép khi tách từ bằng thư viện underthesea:', underthesea_compound_words)
underthesea_sentences[10:15]

Số lượng từ ghép khi tách từ bằng thư viện underthesea: 280


['Lên trên này thấy các em , anh mới thấy mình quá may_mắn .',
 'Mặt_trời vàng như trứng chiên .',
 'Khi mà em đứng_thẳng , núi và đồi phải đứng nghiêng .',
 'Con đường quanh_co quá , em chao lượn như cánh chim .',
 'Mai sau lớn em bơi giữa đời , em bơi khỏe như Ánh_Viên .']

##Tách từ thủ công

In [ ]:
with open('/content/final_tokens.txt', 'r', encoding='utf-8') as f:
    final_tokenize_sentences = []
    sentence = ''
    for word in f:
        if word == '\n':
            final_tokenize_sentences.append(sentence.strip())
            sentence = ''
        else: sentence += word.replace('\n', ' ')
final_tokenize_compound_words = 0
for sentence in final_tokenize_sentences:
    for word in sentence.split():
        if '_' in word: final_tokenize_compound_words += 1
print('Số lượng từ ghép khi tách từ thủ công:', final_tokenize_compound_words)
final_tokenize_sentences[10:15]

Số lượng từ ghép khi tách từ thủ công: 267


['lên trên này thấy các em , anh mới thấy mình quá may_mắn .',
 'mặt_trời vàng như trứng chiên .',
 'khi mà em đứng thẳng , núi và đồi phải đứng nghiêng .',
 'con đường quanh_co quá , em chao lượn như cánh chim .',
 'mai_sau lớn em bơi giữa đời , em bơi khỏe như Ánh_Viên .']

##Đánh giá


In [ ]:
def count_correct_words(pred, source, n_grams=3):
    pred_words = pred.split()
    source_words = source.split()
    total_true = 0
    true_positive = 0
    total_errors = 0
    false_positive = 0
    index = 0
    while index < len(pred_words):
        if pred_words[index] not in source_words[index:(index + n_grams)]:
            if '_' in pred_words[index]:
                false_positive += 1
            del pred_words[index]
            total_errors += 1
        else:
            index += 1
    index = 0
    while index < len(source_words):
        if source_words[index] not in pred_words[index:(index + n_grams)]:
            del source_words[index]
        else:
            index += 1
    words = pred_words if len(pred_words) < len(source_words) else source_words
    for index in range(len(words)):
        if pred_words[index] == source_words[index]:
            if '_' in pred_words[index]:
                true_positive += 1
            total_true += 1
    return total_true, total_errors, true_positive, false_positive

In [ ]:
def tokenize_evaluation(pred, source, n_grams=3):
    total_true = 0
    total_errors = 0
    total_words = 0
    pred_tp = 0
    pred_fp = 0
    for pred_sentence, source_sentence in zip(pred, source):
        total_words += len(source_sentence.split())
        if pred_sentence != source_sentence:
            result = count_correct_words(pred_sentence, source_sentence, n_grams)
            total_true += result[0]
            total_errors += result[1]
            pred_tp += result[2]
            pred_fp += result[3]
        else:
            for word in source_sentence.split():
                if '_' in word:
                    pred_tp += 1
                total_true += 1
    return {
        'Accuracy': total_true / total_words,
        'Precision': pred_tp / (pred_tp + pred_fp),
        'Recall': pred_tp / final_tokenize_compound_words,
    }

In [ ]:
longest_matching_evaluation = tokenize_evaluation(longest_matching_sentences, final_tokenize_sentences)
underthesea_evaluation = tokenize_evaluation(underthesea_sentences, final_tokenize_sentences)
pd.DataFrame([longest_matching_evaluation, underthesea_evaluation],index = ['Longest Matching','Underthesea']).astype(object).T

,Longest Matching,Underthesea
Accuracy,0.587102,0.743719
Precision,0.505435,0.653285
Recall,0.348315,0.670412
